In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra"
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("/kaggle/input/finaltest/df_final_clean.csv")

In [14]:
df.columns

Index(['index', 'image_link', 'group_id', 'entity_name', 'text',
       'cleaned_column', 'val_with_unit', 'item_vol_unit'],
      dtype='object')

In [15]:
df.drop(columns=[ 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15','Unnamed: 0'],inplace=True)

KeyError: "['Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 0'] not found in axis"

In [ ]:
import re
def extract_numerical_value(text):
    # Pattern to match numbers (including decimals) potentially followed by units
    pattern = r'\b(\d+(?:\.\d+)?)\s*([a-zA-Z]*)\b'
    matches = re.findall(pattern, text)
    
    if matches:
        return [(float(value), unit) for value, unit in matches]
    return None

df['val_with_unit']=df['text'].apply(extract_numerical_value)

In [ ]:
def extract_numerical_vol(text):
    # Pattern to match numbers followed immediately by letters
    pattern = r'(\d+(?:\.\d+)?)([A-Za-z]{1,2})'
    matches = re.findall(pattern, text)
    result = [(float(num), unit[:2].upper()) for num, unit in matches]
    
    return result


df['item_vol_unit']=df['text'].apply(extract_numerical_vol)

In [ ]:
df['val_with_unit'] = df['val_with_unit'].combine_first(df['item_vol_unit'])

In [ ]:
df.iloc[402]['image_link']

In [16]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart'}
}


In [17]:
synonyms = {'millimetre': {'', 'mm','MM','MILLIMETRE'},
 'yard': {'','yards'},
 'metre': {'m','M'},
 'centimetre': {'', 'CM', 'cm','cms','Cms','centimeter','CMS'},
 'inch': {'', 'in', 'inch','inchs','inches','Inches','INCHES','In'},
 'foot': {'','Ft','FEET','FT','FooT'},
 'gram': {'',
  
  'G',
  'GA',
  'GAME',
  'GAMEBOARDS',
  'GAMEar',
  'GAUGE',
  'GEM',
  'GM',
  'GMS',
  'GORM',
  'GPM',
  'GPROTEIN',
  'GR',
  'GRAD',
  'GRAI',
  'GRAM',
  'GRAMGOLD',
  'GRAMM',
  'GRAMS',
  'GRAU',
  'GREMOTE',
  'GRF',
  'GRMS',
  'GSM',
  'GSMDouble',
  'GSMWhite',
  'GTASTE',
  'Game',
  'Gaurav',
  'Glasbreite',
  'Globally',
  'Gm',
  'Gms',
  'Gr',
  'Grac',
  'Grad',
  'GradMaschinenwaschbar',
  'GraddrehbareSchnalle',
  'Gram',
  'Gramm',
  'Gramme',
  'Gramo',
  'Grams',
  'Gramsof',
  'Grau',
  'GsM',
  'Gura',
  'NET',
  'g',
  'gAbout',
  'gC',
  'gCAT',
  'gDC',
  'gE',
  'gEach',
  'gFREE',
  'gInosivarentsprecheninetwaeinemgestrichenen',
  'gMADENVEN',
  'gNet',
  'gXL',
  'gZutaten',
  'galways',
  'game',
  'garound',
  'gayord',
  'ge',
  'geC',
  'ggyFg',
  'ggyard',
  'ggyurd',
  'gm',
  'gms',
  'gn',
  'gnet',
  'gpack',
  'gpendant',
  'gr',
  'gram',
  'gramm',
  'grammes',
  'grammi',
  'grams',
  'groms',
  'grus',
  'gsM',
  'gsize',
  'gsm',
  'gt',
  'gure',
  'gx',
  
  'zu'},
 'ounce': {'',
 
  'LB',
  'LEMON',
  'MEZCOTOYZ',
  'O',
  'OICE',
  'ONE',
  'OUNCES',
  'OZ',
  'OZBOILING',
  'One',
  'Onz',
  'Ounces',
  'Oz',
  'ounce'
  'o',
  'oZ',
  'onces',
  'one',
  'ounces',
  'oz',
  },
 'milligram': {'',
  
  'MG',
  'MGMELATONIN',
  'MGserving',
  'MGt',
  'MV',
  'Mg',
  'Milligramm',
  'PER',
 
  'capsules',
  'mcg',
  'mg',
  'mgKieselerde',
  'mgRef',
  'mgserving',
  'mgt',
  'mng',
  's'},
 'microgram': {'', 'MCG', 'POP', 'adw', 'mcg', 'mcgvitamin', 'µg'},
 'kilogram': {'',
 
  'COMPOSTABLE',
  'Capacity',
   'Okg'
  'K',
  'KG',
  'KGP',
  'KGS',
  'KGWeight',
  'KGand',
  'KGe',
  'KGs',
  'KILOGRAM',
  'KT',
  'Ka',
  'Kg',
  'Kgs',
  'Kilo',
  'Kilograms',
  'MAX',
  'MG',
  'Min',
  'N',
  'NW',
  'PORTION',
  'PROFESSIONAL',
  'RG',
  'Required',
  'SCOOp',
  'Scoops',
  'TRITTICO',
  'WEIGHTBEARING',
  'Weight',
 
  'h',
  'ig',
  'io',
  'k',
  'kG',
  'kGe',
  'kc',
  'kcal',
  'kg',
  'kgC',
  'kgMax',
  'kgbody',
  'kge',
  'kgr',
  'kgs',
  'kgyou',
  'ko',
  'kq',
 },
'ton': {'',
  
  'TO',
  'To',
  
  't',
  'to'},
 'pound': {'',
  'bs',
  'Ib',
  'Ibs',
  'Kuhlakkus',
  'L',
  'LB',
  'LBS',
  'LBSTENSION',
  'LBs',
  'LEE',
  'Lagen',
  'Lb',
  'Lbs',
  'Lebenspunkt',
  'Lichtmodi',
  'Lost',
  'ND',
  'Nr',
  'PACK',
  'POUND',
  'POUNDS',
    'lbs',
  'Pound',
  'Pounds',
     'LBS',
  'lb',
     'Lb',
  'pounds','lbs'
 },
  
 'gallon': {'', 'gal','Gallon','GALLON','Gallo','Gallons','GALLONS','gallons','Gallon','gallon'},
 'watt': {'',
  'watt',
  'W',
  'WATT',
  'WATTS',
  'WBrushless',
  'WElectric',
  'WMAX',
  'WPOWER',
  'WUSB',
  'Watt',
  'Watts',
  'kWh',
  'levels',
  'r','V','v','M',
  'seconds',
  'w',
  'wCharging',
  'watts'},
 'kilowatt': {'', 'Cost', 'KW', 'Kw', 'kW', 'kWh','kwh','KWH'},
 'volt': {'',
 
  'V','M',
  'VDC',
  'VLED',
  'VOLTS',
  'VRVPOWER',
  'VUSB',
  'Vand',
  'Volt',
  'Volts','volts','A',
  'Vsolar',
  'v','W','w',
    'volt','mAh','Wh',
  'volte'},
 'millivolt': {'mV','mv'},
 'kilovolt': {'kV', 'kv','kv','KV'},
 'litre': {'',
  'L',
  'Liter',
  'Liters',
  'Ltr',
  'liter',
  'litri',
  'lt',
  'l','ltr','litres','Litors','LITERS','liters'},
 'millilitre': {'',
  'MD',
  'ML',
  'Mal',
  'Milliliters',
  'milliliters',
   'milliliter',
  'mI',
  'mL',
  'mLe',
  'mle',
  'mal',
  'ml',
  'mlWater',
  'mlof','MLCONTAINS'},
 'microlitre': {'µL'},
 'decilitre': {'', 'dl'},
 'imperial gallon': {'','imp gal'},
 'centilitre': {'','CL', 'cl'},
 'cup': {'cup','CUP', 'cp'},
 'cubic inch': {'','in³','in','INCHES','inches','inch','Inch'},
 'cubic foot': {'','ft³','FOOT','foot','Foot','ft','FT','CUBICFEET','CubiC','cubic','Cubic'},
 'fluid ounce': {'fl oz','floz','fl','FL','fl','FLOZ','ILoZ','FL OZ','OZ','Fluid Ounces','oz','foz','Ounces','f','F','fluid','eloz','loz','fo','flz','iloz'},
 'pint': {'', 'Opt', 'pt','opt'},
 'quart': {'', 'PATCHES', 'qT', 'qt','Quarts','Quart','quart','quarts'}}

In [18]:
hierarchy = {
    'width': ['centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'],
    'depth': ['centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'],
    'height': ['centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'],
    'item_weight': ['kilogram', 'gram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'],
    'maximum_weight_recommendation': ['kilogram', 'gram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'],
    'voltage': ['volt','kilovolt', 'millivolt'],
    'wattage': ['watt','kilowatt'],
    'item_volume': ['litre', 'millilitre', 'fluid ounce', 'microlitre', 'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'gallon', 'imperial gallon', 'pint', 'quart']
}


In [19]:
df.iloc[401]

index                                                           401
image_link        https://m.media-amazon.com/images/I/31E4X+OlLa...
group_id                                                     557758
entity_name                                             item_volume
text                                                  ['13FO335m)']
cleaned_column                                             13FO335m
val_with_unit                              [(13.0, FO), (335.0, M)]
item_vol_unit                              [(13.0, FO), (335.0, M)]
Name: 401, dtype: object

In [20]:
df.iloc[3315]

index                                                          3321
image_link        https://m.media-amazon.com/images/I/415ETd3iUi...
group_id                                                     469317
entity_name                           maximum_weight_recommendation
text              ['Storage Capacity.3L', 'Load Capacity:15kg', ...
cleaned_column    Storage Capacity.3L Load Capacity:15kg Folded ...
val_with_unit     [(3.0, L), (15.0, kg), (290.0, ), (190.0, ), (...
item_vol_unit     [(3.0, L), (15.0, KG), (40.0, MM), (1.6, IN), ...
Name: 3315, dtype: object

In [21]:
in_order={
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
}

In [22]:
a=df[df['entity_name']=='height']

In [23]:
df.iloc[647]

index                                                           647
image_link        https://m.media-amazon.com/images/I/31PPFTL04V...
group_id                                                     416664
entity_name                                                  height
text                                        ['10.2"(H)', '7.7"(L)']
cleaned_column                                         10.2"H 7.7"L
val_with_unit                                   [(10.2, ), (7.7, )]
item_vol_unit                                                    []
Name: 647, dtype: object

In [24]:
def correct_and_impute_unit(unit, entity_name):
    if not unit:
        return None  # Remove entries with null units
    
    # Check if the unit matches directly
    if unit in synonyms.get(entity_name, {}):
        return unit
    
    # Iterate through the hierarchy for the specified entity
    for valid_unit in hierarchy.get(entity_name, []):
        if unit in synonyms.get(valid_unit, {}):
            return valid_unit
    
    # If no valid unit is found, remove the entry
    return None

# Function to process each row of measurements
def process_measurements(measurements, entity_name):
    corrected_measurements = []
    for value, unit in measurements:
        corrected_unit = correct_and_impute_unit(unit, entity_name)
        if corrected_unit is not None:
            corrected_measurements.append((value, corrected_unit))
    return corrected_measurements


# Display the cleaned DataFrame
# print(df[['corrected_measurements']])

In [25]:
# a['corrected_measurements'] = a['val_with_unit'].apply(lambda x: process_measurements(x, 'height'))

In [26]:
def get_highest_hierarchy_unit(units, entity_name):
    valid_units = hierarchy.get(entity_name, [])
    # Reverse the hierarchy to get highest priority first    
    for unit in valid_units:
        if unit in units:
            return unit
    return None

# Function to filter measurements by highest hierarchy unit
def filter_by_highest_hierarchy(measurements, entity_name):
    units = [unit for _, unit in measurements]
    highest_unit = get_highest_hierarchy_unit(units, entity_name)
    
    if highest_unit:
        # Filter measurements to include only those with the highest hierarchy unit
        return [(value, unit) for value, unit in measurements if unit == highest_unit]
    return []

# a['filtered_measurements'] = a['corrected_measurements'].apply(lambda x: filter_by_highest_hierarchy(x, 'height'))


In [27]:
# a['corrected_measurements'] = a['val_with_unit'].apply(lambda x: process_measurements(x, 'height'))
# a['filtered_measurements'] = a['corrected_measurements'].apply(lambda x: filter_by_highest_hierarchy(x, 'height'))
# a['max_value_string'] = a['filtered_measurements'].apply(find_max_value_and_format)


In [28]:
def find_max_value_and_format(measurements):
    if not measurements:
        return None
    # Find the maximum value and its unit
    max_value, max_unit = max(measurements, key=lambda x: x[0])
    # Format the output as a string
    return f"{max_value} {max_unit}"
# a['max_value_string'] = a['filtered_measurements'].apply(find_max_value_and_format)


In [29]:
# entity_types = ['height']

# for entity in entity_types:
#     # Filter the dataframe for the current entity
#     a = df[df['entity_name'] == entity]
    
#     # Apply the operations
#     a['corrected_measurements'] = a['val_with_unit'].apply(lambda x: process_measurements(x, entity))
#     a['filtered_measurements'] = a['corrected_measurements'].apply(lambda x: filter_by_highest_hierarchy(x, entity))
#     a['max_value_string'] = a['filtered_measurements'].apply(find_max_value_and_format)
    
#     # Update the original dataframe with the results
#     df.loc[df['entity_name'] == entity, ['corrected_measurements', 'filtered_measurements', 'max_value_string']] = a[['corrected_measurements', 'filtered_measurements', 'max_value_string']]

In [30]:
def find_second_highest_value_and_format(measurements):
    if not measurements:
        return None
    
    # Sort measurements in descending order
    sorted_measurements = sorted(measurements, key=lambda x: x[0], reverse=True)
    
    # Return the highest value if less than two measurements exist
    highest_value, highest_unit = sorted_measurements[0]
    if len(sorted_measurements) < 2:
        return f"{highest_value} {highest_unit}"
    
    # Get the second-highest value and its unit
    second_highest_value, second_highest_unit = sorted_measurements[1]
    
    # Format the output as a string
    return f"{second_highest_value} {second_highest_unit}"




In [31]:
# Ensure the 'ans' column exists by initializing it with empty sets
df['ans'] = ["" for _ in range(len(df))]

# Iterate over the DataFrame rows
for idx, row in df.iterrows():
    found = False
    
    # Check if 'val_with_unit' is not None before trying to iterate over it
    if row['val_with_unit'] is not None:
        for value, unit in row['val_with_unit']:
            if not unit.strip() :
                continue
            unit=unit.lower()
            entity_name = row['entity_name']
            for target_unit in hierarchy.get(entity_name, []):
                if target_unit in synonyms:
                        if unit in synonyms[target_unit]:
                            df.at[idx, 'ans']= f"{value} {target_unit}"
                            found = True
                            break
                if found:
                    break
                    
            if found:
                break
    

        if not found:
            df.at[idx, 'ans'] ="" 
        

In [32]:
entity_types = ['height', 'width']

for entity in entity_types:
    # Filter the dataframe for the current entity
    a = df[df['entity_name'] == entity]
    
    # Apply the operations
    a['corrected_measurements'] = a['val_with_unit'].apply(lambda x: process_measurements(x, entity))
    a['filtered_measurements'] = a['corrected_measurements'].apply(lambda x: filter_by_highest_hierarchy(x, entity))
    
    # Use different functions for height and width
    if entity == 'height':
        a['max_value_string'] = a['filtered_measurements'].apply(find_max_value_and_format)
    elif entity == 'width':
        a['max_value_string'] = a['filtered_measurements'].apply(find_second_highest_value_and_format)
    
    # Update the original dataframe with the results
    df.loc[df['entity_name'] == entity, ['corrected_measurements', 'filtered_measurements', 'max_value_string']] = a[['corrected_measurements', 'filtered_measurements', 'max_value_string']]

/tmp/ipykernel_36/3530197992.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['corrected_measurements'] = a['val_with_unit'].apply(lambda x: process_measurements(x, entity))
/tmp/ipykernel_36/3530197992.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['filtered_measurements'] = a['corrected_measurements'].apply(lambda x: filter_by_highest_hierarchy(x, entity))
/tmp/ipykernel_36/3530197992.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [35]:
df['ans'] = df['max_value_string'].fillna(df['ans'])

In [36]:
df

,index,image_link,group_id,entity_name,text,cleaned_column,val_with_unit,item_vol_unit,ans,corrected_measurements,filtered_measurements,max_value_string
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,"['2.63in', '6.68cm', '91.44cm - 199.39cm', '36...",2.63in 6.68cm 91.44cm - 199.39cm 36in - 78in,"[(2.63, in), (6.68, cm), (91.44, cm), (199.39,...","[(2.63, IN), (6.68, CM), (91.44, CM), (199.39,...",199.39 centimetre,"[(2.63, inch), (6.68, centimetre), (91.44, cen...","[(6.68, centimetre), (91.44, centimetre), (199...",199.39 centimetre
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,"['42cm/16.54""', '200cm/78.74""']","42cm/16.54"" 200cm/78.74""","[(42.0, cm), (16.54, ), (200.0, cm), (78.74, )]","[(42.0, CM), (200.0, CM)]",42.0 centimetre,"[(42.0, centimetre), (200.0, centimetre)]","[(42.0, centimetre), (200.0, centimetre)]",42.0 centimetre
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,"['42cm/16.54""', '200cm/78.74""']","42cm/16.54"" 200cm/78.74""","[(42.0, cm), (16.54, ), (200.0, cm), (78.74, )]","[(42.0, CM), (200.0, CM)]",200.0 centimetre,"[(42.0, centimetre), (200.0, centimetre)]","[(42.0, centimetre), (200.0, centimetre)]",200.0 centimetre
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,"['42cm/16.54""', '200cm/78.74""']","42cm/16.54"" 200cm/78.74""","[(42.0, cm), (16.54, ), (200.0, cm), (78.74, )]","[(42.0, CM), (200.0, CM)]",42.0 centimetre,NaN,NaN,NaN
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,"['10.50cm/4.13""', '90cm/35.43""']","10.50cm/4.13"" 90cm/35.43""","[(10.5, cm), (4.13, ), (90.0, cm), (35.43, )]","[(10.5, CM), (90.0, CM)]",10.5 centimetre,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation,"['1,500 LB weight capacity']",1500 LB weight capacity,"[(1.0, ), (500.0, LB)]",[],500.0 pound,NaN,NaN,NaN
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight,[],NaN,[],[],,NaN,NaN,NaN
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation,[],NaN,[],[],,NaN,NaN,NaN
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight,['955123-14567'],955123-14567,"[(955123.0, ), (14567.0, )]",[],,NaN,NaN,NaN


In [37]:
df['empty_unit_result'] = None

# Function to process each row
def process_empty_units(row):
    if row['entity_name'] in ['height', 'width'] and row['val_with_unit'] is not None:
        for value, unit in row['val_with_unit']:
            if not unit.strip():  # Check if the unit is empty
                return f"{value} inch"
    return None

# Apply the function to each row
df['empty_unit_result'] = df.apply(process_empty_units, axis=1)

# Merge 'empty_unit_result' into 'ans' where conditions are met
df['ans'] = df.apply(lambda row: row['empty_unit_result'] if row['ans'] == "" and row['empty_unit_result'] is not None else row['ans'], axis=1)


In [38]:
df

,index,image_link,group_id,entity_name,text,cleaned_column,val_with_unit,item_vol_unit,ans,corrected_measurements,filtered_measurements,max_value_string,empty_unit_result
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,"['2.63in', '6.68cm', '91.44cm - 199.39cm', '36...",2.63in 6.68cm 91.44cm - 199.39cm 36in - 78in,"[(2.63, in), (6.68, cm), (91.44, cm), (199.39,...","[(2.63, IN), (6.68, CM), (91.44, CM), (199.39,...",199.39 centimetre,"[(2.63, inch), (6.68, centimetre), (91.44, cen...","[(6.68, centimetre), (91.44, centimetre), (199...",199.39 centimetre,None
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,"['42cm/16.54""', '200cm/78.74""']","42cm/16.54"" 200cm/78.74""","[(42.0, cm), (16.54, ), (200.0, cm), (78.74, )]","[(42.0, CM), (200.0, CM)]",42.0 centimetre,"[(42.0, centimetre), (200.0, centimetre)]","[(42.0, centimetre), (200.0, centimetre)]",42.0 centimetre,16.54 inch
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,"['42cm/16.54""', '200cm/78.74""']","42cm/16.54"" 200cm/78.74""","[(42.0, cm), (16.54, ), (200.0, cm), (78.74, )]","[(42.0, CM), (200.0, CM)]",200.0 centimetre,"[(42.0, centimetre), (200.0, centimetre)]","[(42.0, centimetre), (200.0, centimetre)]",200.0 centimetre,16.54 inch
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,"['42cm/16.54""', '200cm/78.74""']","42cm/16.54"" 200cm/78.74""","[(42.0, cm), (16.54, ), (200.0, cm), (78.74, )]","[(42.0, CM), (200.0, CM)]",42.0 centimetre,NaN,NaN,NaN,None
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,"['10.50cm/4.13""', '90cm/35.43""']","10.50cm/4.13"" 90cm/35.43""","[(10.5, cm), (4.13, ), (90.0, cm), (35.43, )]","[(10.5, CM), (90.0, CM)]",10.5 centimetre,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation,"['1,500 LB weight capacity']",1500 LB weight capacity,"[(1.0, ), (500.0, LB)]",[],500.0 pound,NaN,NaN,NaN,None
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight,[],NaN,[],[],,NaN,NaN,NaN,None
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation,[],NaN,[],[],,NaN,NaN,NaN,None
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight,['955123-14567'],955123-14567,"[(955123.0, ), (14567.0, )]",[],,NaN,NaN,NaN,None


In [39]:
df.to_csv("part4.csv")

In [ ]:
df['new_col']

In [76]:
df['ans'] = df['max_value_string'].fillna(df['ans'])

In [78]:
df

,index,image_link,group_id,entity_name,text,cleaned_column,val_with_unit,item_vol_unit,ans,corrected_measurements,filtered_measurements,max_value_string
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,"['2.63in', '6.68cm', '91.44cm - 199.39cm', '36...",2.63in 6.68cm 91.44cm - 199.39cm 36in - 78in,"[(2.63, in), (6.68, cm), (91.44, cm), (199.39,...","[(2.63, IN), (6.68, CM), (91.44, CM), (199.39,...",199.39 centimetre,"[(2.63, inch), (6.68, centimetre), (91.44, cen...","[(6.68, centimetre), (91.44, centimetre), (199...",199.39 centimetre
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,"['42cm/16.54""', '200cm/78.74""']","42cm/16.54"" 200cm/78.74""","[(42.0, cm), (16.54, ), (200.0, cm), (78.74, )]","[(42.0, CM), (200.0, CM)]",42.0 centimetre,"[(42.0, centimetre), (200.0, centimetre)]","[(42.0, centimetre), (200.0, centimetre)]",42.0 centimetre
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,"['42cm/16.54""', '200cm/78.74""']","42cm/16.54"" 200cm/78.74""","[(42.0, cm), (16.54, ), (200.0, cm), (78.74, )]","[(42.0, CM), (200.0, CM)]",200.0 centimetre,"[(42.0, centimetre), (200.0, centimetre)]","[(42.0, centimetre), (200.0, centimetre)]",200.0 centimetre
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,"['42cm/16.54""', '200cm/78.74""']","42cm/16.54"" 200cm/78.74""","[(42.0, cm), (16.54, ), (200.0, cm), (78.74, )]","[(42.0, CM), (200.0, CM)]",42.0 centimetre,NaN,NaN,NaN
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,"['10.50cm/4.13""', '90cm/35.43""']","10.50cm/4.13"" 90cm/35.43""","[(10.5, cm), (4.13, ), (90.0, cm), (35.43, )]","[(10.5, CM), (90.0, CM)]",10.5 centimetre,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation,"['1,500 LB weight capacity']",1500 LB weight capacity,"[(1.0, ), (500.0, LB)]",[],500.0 pound,NaN,NaN,NaN
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight,[],NaN,[],[],,NaN,NaN,NaN
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation,[],NaN,[],[],,NaN,NaN,NaN
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight,['955123-14567'],955123-14567,"[(955123.0, ), (14567.0, )]",[],,NaN,NaN,NaN


In [33]:
df.to_csv('dict_change.csv')

,index,image_link,group_id,entity_name,text,cleaned_column,val_with_unit,item_vol_unit,corrected_measurements,filtered_measurements,max_value_string,ans
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,"['2.63in', '6.68cm', '91.44cm - 199.39cm', '36...",2.63in 6.68cm 91.44cm - 199.39cm 36in - 78in,"[(2.63, in), (6.68, cm), (91.44, cm), (199.39,...","[(2.63, IN), (6.68, CM), (91.44, CM), (199.39,...","[(2.63, inch), (6.68, centimetre), (91.44, cen...","[(6.68, centimetre), (91.44, centimetre), (199...",199.39 centimetre,199.39 centimetre
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,"['42cm/16.54""', '200cm/78.74""']","42cm/16.54"" 200cm/78.74""","[(42.0, cm), (16.54, ), (200.0, cm), (78.74, )]","[(42.0, CM), (200.0, CM)]","[(42.0, centimetre), (200.0, centimetre)]","[(42.0, centimetre), (200.0, centimetre)]",42.0 centimetre,42.0 centimetre
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,"['42cm/16.54""', '200cm/78.74""']","42cm/16.54"" 200cm/78.74""","[(42.0, cm), (16.54, ), (200.0, cm), (78.74, )]","[(42.0, CM), (200.0, CM)]","[(42.0, centimetre), (200.0, centimetre)]","[(42.0, centimetre), (200.0, centimetre)]",200.0 centimetre,200.0 centimetre
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,"['42cm/16.54""', '200cm/78.74""']","42cm/16.54"" 200cm/78.74""","[(42.0, cm), (16.54, ), (200.0, cm), (78.74, )]","[(42.0, CM), (200.0, CM)]",NaN,NaN,NaN,42.0 centimetre
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,"['10.50cm/4.13""', '90cm/35.43""']","10.50cm/4.13"" 90cm/35.43""","[(10.5, cm), (4.13, ), (90.0, cm), (35.43, )]","[(10.5, CM), (90.0, CM)]",NaN,NaN,NaN,10.5 centimetre
...,...,...,...,...,...,...,...,...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation,"['1,500 LB weight capacity']",1500 LB weight capacity,"[(1.0, ), (500.0, LB)]",[],NaN,NaN,NaN,500.0 pound
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight,[],NaN,[],[],NaN,NaN,NaN,
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation,[],NaN,[],[],NaN,NaN,NaN,
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight,['955123-14567'],955123-14567,"[(955123.0, ), (14567.0, )]",[],NaN,NaN,NaN,


In [193]:
# df.to_csv("hierarchy.csv")

In [218]:
df.loc[df['entity_name']=='depth','ans']=''

In [221]:
# df.to_csv("hierarchy1.csv")

In [223]:
df.columns

Index(['index', 'image_link', 'group_id', 'entity_name', 'text',
       'cleaned_column', 'val_with_unit', 'item_vol_unit',
       'corrected_measurements', 'filtered_measurements', 'max_value_string',
       'ans'],
      dtype='object')

In [162]:
df[df['text']=='[]']

,index,image_link,group_id,entity_name,text,cleaned_column,val_with_unit,ans
15,15,https://m.media-amazon.com/images/I/216rjgJHAe...,279307,item_weight,[],NaN,None,
16,16,https://m.media-amazon.com/images/I/2174yonQBt...,569206,maximum_weight_recommendation,[],NaN,None,
21,21,https://m.media-amazon.com/images/I/21CxlWbim3...,751532,width,[],NaN,None,
56,56,https://m.media-amazon.com/images/I/21qleENrBB...,953313,item_weight,[],NaN,None,
80,80,https://m.media-amazon.com/images/I/31+KMtY+QX...,630390,item_weight,[],NaN,None,
...,...,...,...,...,...,...,...,...
131165,131266,https://m.media-amazon.com/images/I/A1KuAtMexv...,603688,item_weight,[],NaN,None,
131166,131267,https://m.media-amazon.com/images/I/A1KuAtMexv...,603688,maximum_weight_recommendation,[],NaN,None,
131178,131279,https://m.media-amazon.com/images/I/A1kL4i+kcf...,257505,item_volume,[],NaN,None,
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight,[],NaN,None,


In [163]:
df.iloc[21]['image_link']

'https://m.media-amazon.com/images/I/21CxlWbim3L.jpg'

In [160]:
df.to_csv("my_predictions.csv")

In [140]:
df.iloc[130472]['image_link']

'https://m.media-amazon.com/images/I/91GgIFuxciL.jpg'

In [144]:
def tuple_to_string(tuple_value):
    """Converts a tuple (value, unit) to a string."""
    return f"{tuple_value[0]} {tuple_value[1]}"

df['weight_string'] = df['ans'].apply(tuple_to_string)

TypeError: 'set' object is not subscriptable

In [93]:
df.iloc[123]['image_link']

'https://m.media-amazon.com/images/I/310PNQ4FmqL.jpg'

In [102]:
if df.iloc[131182]['val_with_unit'] is not None:
    for value, unit in df.iloc[131182]['val_with_unit']:
        entity_name=df.iloc[131182]['entity_name']
        if not unit.strip():  
            continue
        if entity_name in entity_unit_map:
            for target_unit in hierarchy[entity_name]:
                for target_unit in synonyms:
                    if unit in synonyms[target_unit]:
                        print(target_unit)
                        found=True
                        break
                        
                if found:
                    break
        
        if found:
            break

pound


In [43]:
a=df[df['entity_name']=='height']

In [45]:
df

,index,image_link,group_id,entity_name,text,cleaned_column,val_with_unit,item_vol_unit
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,"['2.63in', '6.68cm', '91.44cm - 199.39cm', '36...",2.63in 6.68cm 91.44cm - 199.39cm 36in - 78in,"[(2.63, in), (6.68, cm), (91.44, cm), (199.39,...","[(2.63, IN), (6.68, CM), (91.44, CM), (199.39,..."
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,"['42cm/16.54""', '200cm/78.74""']","42cm/16.54"" 200cm/78.74""","[(42.0, cm), (16.54, ), (200.0, cm), (78.74, )]","[(42.0, CM), (200.0, CM)]"
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,"['42cm/16.54""', '200cm/78.74""']","42cm/16.54"" 200cm/78.74""","[(42.0, cm), (16.54, ), (200.0, cm), (78.74, )]","[(42.0, CM), (200.0, CM)]"
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,"['42cm/16.54""', '200cm/78.74""']","42cm/16.54"" 200cm/78.74""","[(42.0, cm), (16.54, ), (200.0, cm), (78.74, )]","[(42.0, CM), (200.0, CM)]"
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,"['10.50cm/4.13""', '90cm/35.43""']","10.50cm/4.13"" 90cm/35.43""","[(10.5, cm), (4.13, ), (90.0, cm), (35.43, )]","[(10.5, CM), (90.0, CM)]"
...,...,...,...,...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation,"['1,500 LB weight capacity']",1500 LB weight capacity,"[(1.0, ), (500.0, LB)]",[]
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight,[],NaN,[],[]
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation,[],NaN,[],[]
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight,['955123-14567'],955123-14567,"[(955123.0, ), (14567.0, )]",[]


In [98]:
if df.iloc[131182]['val_with_unit'] is not None:
    for value, unit in df.iloc[131182]['val_with_unit']:
        entity_name=df.iloc[131182]['entity_name']
        if not unit.strip():  
            continue
        if entity_name in entity_unit_map:
            for target_unit in hierarchy[entity_name]:
                for target_unit in synonyms:
                    if unit in synonyms[target_unit]:
                        print(target_unit)
                        found=True
                        break
                        
                if found:
                    break
        
        if found:
            break

In [100]:
if not unit.strip():
    print("sijnvc")

sijnvc


In [ ]:
# mport re
# import pandas as pd

# # Define the function to find numbers with the next word
# def find_numbers_with_next_word(input_list, entity_names):
#     results = []
#     pattern = re.compile(r'(\d+(?:\.\d+)?)([ /"\']?)([a-zA-Z"\']*)')
    
#     for item in input_list:
#         matches = pattern.findall(item)
        
#         for match in matches:
#             number, separator, next_word = match
#             next_word_cleaned = next_word.strip().lower()  # Convert to lower case for matching
#             if next_word_cleaned in entity_names:
#                 results.append((number, next_word_cleaned))
    
#     return results

# # Define the function to find the full unit name
# def find_full_unit_name(unit, synonyms):
#     for full_unit_name, variations in synonyms.items():
#         if unit in variations:
#             return full_unit_name
#     return unit  # Return the unit itself if not found

# # Function to get units for a specific entity name
# def get_units_for_entity(entity_name, unit_variations, synonyms):
#     units = unit_variations.get(entity_name, set())
#     all_synonyms = set()
#     for unit in units:
#         all_synonyms.update(synonyms.get(unit, set()))
#     return list(all_synonyms)

# # Function to process each row of the DataFrame
# def process_row(row):
#     cleaned_column = row['cleaned_column']
    
#     # Ensure that cleaned_column is a list, if not, return None
#     if isinstance(cleaned_column, str):
#         cleaned_column = [cleaned_column]
#     elif not isinstance(cleaned_column, list):
#         return None
    
#     entity_name = row['entity_name']
#     entity_name_units = get_units_for_entity(entity_name, unit_variations, synonyms)
#     result = find_numbers_with_next_word(cleaned_column, entity_name_units)
    
#     # Sort results based on the hierarchy of the entity_name
#     hierarchy = unit_hierarchy.get(entity_name, [])
#     if not hierarchy:
#         # If there's no hierarchy, just return the results
#         sorted_results = result
#     else:
#         # Sort results based on the hierarchy
#         sorted_results = sorted(result, key=lambda x: hierarchy.index(find_full_unit_name(x[1], synonyms)) if find_full_unit_name(x[1], synonyms) in hierarchy else len(hierarchy))

#     if sorted_results:
#         number, unit = sorted_results[0]
#         full_unit_name = find_full_unit_name(unit, synonyms)
#         if full_unit_name:
#             return f"{number} {full_unit_name}"
    
#     return None

# # Example DataFrame

# # Apply the function to each row
# df['ans'] = df.apply(process_row, axis=1)


In [36]:
df

,index,image_link,group_id,entity_name,text,cleaned_column,val_with_unit,item_vol_unit
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,"['2.63in', '6.68cm', '91.44cm - 199.39cm', '36...",2.63in 6.68cm 91.44cm - 199.39cm 36in - 78in,"[(2.63, in), (6.68, cm), (91.44, cm), (199.39,...","[(2.63, IN), (6.68, CM), (91.44, CM), (199.39,..."
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,"['42cm/16.54""', '200cm/78.74""']","42cm/16.54"" 200cm/78.74""","[(42.0, cm), (16.54, ), (200.0, cm), (78.74, )]","[(42.0, CM), (200.0, CM)]"
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,"['42cm/16.54""', '200cm/78.74""']","42cm/16.54"" 200cm/78.74""","[(42.0, cm), (16.54, ), (200.0, cm), (78.74, )]","[(42.0, CM), (200.0, CM)]"
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,"['42cm/16.54""', '200cm/78.74""']","42cm/16.54"" 200cm/78.74""","[(42.0, cm), (16.54, ), (200.0, cm), (78.74, )]","[(42.0, CM), (200.0, CM)]"
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,"['10.50cm/4.13""', '90cm/35.43""']","10.50cm/4.13"" 90cm/35.43""","[(10.5, cm), (4.13, ), (90.0, cm), (35.43, )]","[(10.5, CM), (90.0, CM)]"
...,...,...,...,...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation,"['1,500 LB weight capacity']",1500 LB weight capacity,"[(1.0, ), (500.0, LB)]",[]
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight,[],NaN,[],[]
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation,[],NaN,[],[]
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight,['955123-14567'],955123-14567,"[(955123.0, ), (14567.0, )]",[]
